# Redo download

I need to redo all of my downloads, because I had been doing it wrong. So this notebook does that.

1. 

2. Download tars based on metadata.
    - If tars exist on 

In [27]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import boto3, configparser, os

Connect to Google Drive to see which tars we have stored ourselves (to save $):

In [3]:
# Set up Google Drive according to docs
def connect_to_google_drive():
    g_login = GoogleAuth()
    g_login.LocalWebserverAuth()
    drive = GoogleDrive(g_login)
    return drive

drive = connect_to_google_drive()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=205689913441-4qvumj04tvu7o2h0j1cth62qhp0ck9ld.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [18]:
def download_gdrive_file(file, title):
    '''
    Downloads given file from Google Drive.

    Parameters
    ----------
    file : str
        The file in the form of a GoogleFile object 
    title : str
        The filename of the file
    '''
        
    # Ensure src directory exists 
    if not os.path.isdir('src'):
        os.makedirs('src')
        
    # Get Google file
    
    # Download file
    print('Downloading ' + title + '...')
    file.GetContentFile(title) 
    print('Successfully downloaded drive://arxiv/{} to {}'.format(os.path.basename(title), title))
    

def query_tarfiles_from_arxiv_folder_on_gdrive():
    '''
    Gets names of all tarfiles in arxiv folder on Google Drive. 
    Returns list of strings, each string representing a name. 
    If there is no folder or it is empty, an empty list is returned. 
    '''
    
    gdrive_tars = []
    query = "'root' in parents and trashed=false and title='arxiv' and mimeType='application/vnd.google-apps.folder'"
    arxiv_folder_id = drive.ListFile({'q': query}).GetList()[0].metadata['id']
    
    # If folder doesn't exist, create and upload it, exit (we will have to query everything from S3)
    if not arxiv_folder_id:
        print('Google Drive does not contain a folder named \'arxiv\'. Creating folder...')
        arxiv_folder = drive.CreateFile({'title': 'arxiv', 
                                    "mimeType": "application/vnd.google-apps.folder"}) 
        arxiv_folder.Upload()
        arxiv_folder_id = arxiv_folder['id']
        print('Folder created.')
    else:
        # Get list of files in arxiv folder
        gdrive_tars = drive.ListFile({'q': "'" + arxiv_folder_id + "' in parents and trashed=false"}).GetList()
        gdrive_tars_str = [x.metadata['title'] for x in gdrive_tars]
        print('Number of tars on Google Drive: ' + str(len(gdrive_tars)))
        gdrive_tars_str.sort()
    
    return gdrive_tars

In [43]:
gdrive_tars = query_tarfiles_from_arxiv_folder_on_gdrive()

Number of tars on Google Drive: 1388


In [45]:
global s3resource 

def setup():
    '''
    Creates S3 resource & sets configs to enable download.
    '''

    # Securely import configs from private AWS config file
    configs = configparser.ConfigParser()
    configs.read('config.ini')

    # Create S3 resource & set configs
    global s3resource
    s3resource = boto3.resource(
        's3',  # the AWS resource we want to use
        aws_access_key_id=configs['DEFAULT']['ACCESS_KEY'],
        aws_secret_access_key=configs['DEFAULT']['SECRET_KEY'],
        region_name='us-east-1'  # same region the arxiv bucket is in
    )

def query_arxiv_bucket():
    '''
    Queries S3 arxiv bucket to see which tar files to download,
    and calls download_file() on the appropriate tar files. 
    '''

    print('Querying S3 arxiv bucket...')
    setup()

    # Create a reusable Paginator
    paginator = s3resource.meta.client.get_paginator('list_objects_v2')

    # Create a PageIterator from the Paginator
    page_iterator = paginator.paginate(
        Bucket='arxiv',
        RequestPayer='requester',
        Prefix='src/'
    )

    # Download and extract tars
    numFiles = 0
    for page in page_iterator:
        numFiles = numFiles + len(page['Contents'])
        for file in page['Contents']:
            key = file['Key']
            # If current file is a tar 
            if key.endswith('.tar'):
                # If it is in Google Drive, download it from there
                # Skip tar download if we already have it (in Google Drive)
                if any(os.path.basename(key) == x.metadata['title'] for x in gdrive_tars):
                    # Download the gdrive tar
                    # Extract astro-ph submissions 
                    # Remove tar from local storage
                # Otherwise download it from S3
                else:
                    # Download .tar
                    # Extract astro-ph submissions
                    # Remove tar from local storage
                
                    #download_file(key)
                    # Extract astrophysics preprints from the .tar
                    #extract_tar(key)
                    # Upload .tar to Google Drive for storage
                    #upload_to_google_drive(key)
                    # Remove tar from local storage/the computer)
                    #os.remove(key)
                    pass
            
    print('Processed ' + str(numFiles - 1) + ' tars')  # -1 
    
query_arxiv_bucket()

Querying S3 arxiv bucket...
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists

Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in G

Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in Google Drive
Exists in G

Processed 2182 tars
